#Import the newest posts and store data in a separate file

In [22]:
import requests
import pandas as pd
import json

CLIENT_ID = '0SVd1VBTNgtpN7b-IA31Dw'
SECRET_KEY = 'T5oxXIdKpNTX0Z4Fy0rpyX3MeQFrXg'

auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

with open('pw.txt', 'r') as f:
    username = f.readline()
    pw = f.readline()
    
    
data = {
    'grant_type' : 'password',
    'username' : username,
    'password' : pw
}
headers = {'User-Agent': 'MyAPI/0.0.1'}

res = requests.post('https://www.reddit.com/api/v1/access_token',
                    auth=auth, data=data, headers=headers)
TOKEN = res.json()['access_token']

headers['Authorization'] = f'bearer {TOKEN}'

# Obtain 100 of the newest posts 
params = {
    'limit': 100
    }
res = requests.get('http://oauth.reddit.com/r/umd/new/',
                 headers = headers, params=params)
df = pd.DataFrame()

for post in res.json()['data']['children']:
    df = df._append({
        'title': post['data']['title'],
        'upvotes': post['data']['ups'],
        'downvotes':post['data']['downs'],
        'flair': post['data']['link_flair_text']
        
    }, ignore_index=True)

df.to_csv(path_or_buf='umdtoppostsdata', mode = 'w', index = False)